# 📊 Inventory Demand Forecasting with Prophet
Forecast daily product demand using Facebook Prophet.

In [ ]:
# Install Prophet if not already installed
!pip install prophet

In [ ]:
import pandas as pd
import numpy as np

## 🔧 Generate Sample Sales Data

In [ ]:
date_range = pd.date_range(start="2023-01-01", end="2024-12-31", freq="D")
np.random.seed(42)
data = []
products = ['PROD_A', 'PROD_B', 'PROD_C']
for product in products:
    base_demand = np.random.randint(20, 100)
    seasonal = np.sin(np.linspace(0, 10 * np.pi, len(date_range))) * 10
    noise = np.random.normal(0, 5, len(date_range))
    sales = base_demand + seasonal + noise
    sales = np.maximum(sales, 0).astype(int)
    df = pd.DataFrame({
        'date': date_range,
        'product_id': product,
        'sales': sales
    })
    data.append(df)
sales_data = pd.concat(data, ignore_index=True)
sales_data.head()

## 🔮 Forecast Using Prophet

In [ ]:
from prophet import Prophet
import matplotlib.pyplot as plt
product_df = sales_data[sales_data['product_id'] == 'PROD_A'][['date', 'sales']].copy()
product_df = product_df.rename(columns={'date': 'ds', 'sales': 'y'})
model = Prophet(daily_seasonality=True, yearly_seasonality=True)
model.fit(product_df)
future = model.make_future_dataframe(periods=90)
forecast = model.predict(future)
fig1 = model.plot(forecast)
plt.title("Sales Forecast for PROD_A")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.tight_layout()
plt.show()
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()